In [1]:
import random
import numpy as np
import pandas as pd
import time
import re
import datetime
import os
# import matplotlib.pyplot as plt
# import gensim
# import spacy

In [2]:
df = pd.read_excel('Excel_files/Full_Poem_Dataset_9-30_0.xlsx')
print(len(df))
df = df.drop('Unnamed: 0',axis=1)
df.drop_duplicates(subset=['Text'])
print(len(df))
records = df.to_dict('records')

df.head()

3144
3144


,Text,Author,Before or after,Source
0,"Зима в Баварии похожа\nна всякую другую зиму, ...",Cтаніслав Перфецький,Before,Facebook
1,***\n \nгоды \nналево\nходил\nза красным вином...,Maddalena Russa,Before,metajournal
2,\n \nСОРНАЯ СКАЗКА\n \nНадышалась крапива озон...,Unknown,Before,metajournal
3,а он будет долбить и долбить её\nна заднем сид...,Unknown,Before,metajournal
4,"Верю в покойных своей страны, в их подъяремные...",Unknown,Before,metajournal


In [4]:
def initializeRec(recText):
    if isinstance(recText, str):
        return recText.strip()
    else:
        return False

def skipLine(line, idx):
    if len(line.strip()) == 0:
        return False
    
    # throw hashtag line
    if line.strip()[0] == '#':
        return True
    
    # throw attribution line
    for attr in ['из личного','личный блог','источник:','авторский блог']:
        if attr in line.lower():
            return True
        
    # dots at the beginning
    matches = re.search("[\*\+\^-_][*+^-_= ]+", line.strip()) 
    if matches:
        return True
    
    # is none of it alphanumeric
    containsAlpha = False
    for char in line:
        if char.isalpha():
            containsAlpha = True
            break
    if not containsAlpha:
        return True
    
    return False


def processRec(rec):
    recText = initializeRec(rec['Text'])
    cleanLines = []
    if recText:
        # decide which lines to keep
        lines = recText.split('\n')
        for i, line in enumerate(lines):
            if skipLine(line, i):
                continue
            cleanLines.append(line)
    return '\n'.join(cleanLines)

In [8]:
# for poem by poem basis
boa = ['Before','After']
beforeDf = df[df['Before or after'] == 'Before'].sample(1200)
afterDf = df[df['Before or after'] == 'After'].sample(1200)

newRecords = beforeDf.to_dict('records') + afterDf.to_dict('records')

x = [processRec(r) for r in newRecords]
y = [boa.index(r['Before or after']) for r in newRecords]

# for line by line basis
xLines = []
yLines = []
for r in newRecords:
    s = processRec(r)
    a = boa.index(r['Before or after'])
    for line in s.split('\n\n'):
        xLines.append(line)
        yLines.append(a)

print(len(xLines), len(yLines))
print(len(x), len(y))

5745 5745
2400 2400


In [12]:
sentModelDict = dict()
embeddingDict = dict()

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

model1 = 'paraphrase-multilingual-MiniLM-L12-v2'
model2 = "distiluse-base-multilingual-cased-v1"
model3 = 'paraphrase-multilingual-mpnet-base-v2'

for model in [model1,model2,model3]:
    %time sentModelDict[model] = SentenceTransformer(model)
    %time embeddingDict[model] = sentModelDict[model].encode(x, show_progress_bar=True)
    print('sentence embedding done for', model)

CPU times: user 1.88 s, sys: 1.13 s, total: 3.01 s
Wall time: 9.02 s


Batches:   0%|          | 0/75 [00:00<?, ?it/s]

CPU times: user 6min 25s, sys: 38.9 s, total: 7min 4s
Wall time: 14min 3s
sentence embedding done for paraphrase-multilingual-MiniLM-L12-v2


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/402 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

CPU times: user 14.5 s, sys: 7.69 s, total: 22.2 s
Wall time: 3min 40s


Batches:   0%|          | 0/75 [00:00<?, ?it/s]